In [2]:
import itertools
import pandas as pd
from datasets import load_dataset
import json
from PIL import Image
import os

In [6]:
with open('/workspace/data/dataset_flickr30k.json', 'r') as f:
    json_data = json.load(f)

image_entries = json_data['images']
test_filenames = [image['filename'] for image in image_entries if image['split'] == 'test'][:50]

flickr30k_test = load_dataset("nlphuji/flickr30k", split="test")
test_dataset = flickr30k_test.filter(lambda example: example['filename'] in test_filenames)

print("Filtered test Dataset:")
print(test_dataset)

Filtered test Dataset:
Dataset({
    features: ['image', 'caption', 'sentids', 'split', 'img_id', 'filename'],
    num_rows: 50
})


In [8]:
print(f"Loaded {len(test_dataset)} image-caption entries\n")

print("📌 First 5 entries:")
print(test_dataset.select(range(5)), "\n")

print("📋 Column types:")
print(test_dataset.features)

print("\n🔎 Size")
print((len(test_dataset), len(test_dataset.features)))

Loaded 50 image-caption entries

📌 First 5 entries:
Dataset({
    features: ['image', 'caption', 'sentids', 'split', 'img_id', 'filename'],
    num_rows: 5
}) 

📋 Column types:
{'image': Image(mode=None, decode=True, id=None), 'caption': [Value(dtype='string', id=None)], 'sentids': [Value(dtype='string', id=None)], 'split': Value(dtype='string', id=None), 'img_id': Value(dtype='string', id=None), 'filename': Value(dtype='string', id=None)}

🔎 Size
(50, 6)
